# Locust Load Testing

## Machine requriements

This notebook should be run on a machine with at least 8 Nvidia GPUs, or 16 AWS AI chips. Below we show recommended machines for a range of LLM sizes. 

**Note:** As LLM size grows bigger, it may not fit on a single machine, and cannot be tested using this notebook.

### For Nvidia GPUs

| LLM size      | EC2 instance type |
| ----------- | ----------- |
| Upto 10B      | [g6.48xlarge](https://aws.amazon.com/ec2/instance-types/g6/), [g5.48xlarge](https://aws.amazon.com/ec2/instance-types/g5/)      |
| Upto 70B   | [g6e.48xlarge](https://aws.amazon.com/ec2/instance-types/g6e/), [p4d.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| Upto 405B   | [p5.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/), [p4de.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| > 405B   | [p5e.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/)      |

### For AWS AI Chips

| LLM size      | EC2 instance type |
| ----------- | ----------- |    
| Upto 70B   | [trn1.32xlarge](https://aws.amazon.com/ec2/instance-types/trn1/)       |
| > 70B   | [trn2.48xlarge](https://aws.amazon.com/ec2/instance-types/trn2/)      |

Next, let us verify the machine.


In [ ]:
import subprocess
p = subprocess.run('nvidia-smi --list-gpus | wc -l', 
                   shell=True, check=True, capture_output=True, encoding='utf-8')

device = None
num_device = 0

if p.returncode == 0:
    num_device = int(p.stdout)
    device = "cuda" if num_device > 0 else None

if device is None:
    p = subprocess.run('neuron-ls -j | grep neuron_device | wc -l', 
                       shell=True, check=True, capture_output=True, encoding='utf-8')
    if p.returncode == 0:
        num_device = int(p.stdout)
        device = "neuron" if num_device > 0 else None

assert (device == "cuda" and num_device == 8) or (device == "neuron" and num_device == 16), \
    "Machine must have 8 Nvidia CUDA devices, or 16 AWS Neuorn Devices"
print(f"Auto detected {num_device} {device} devices")

Next, install required Python packages.

In [ ]:
!pip install --upgrade pip
!pip install locust
!pip install datasets
!which locust

### Create Hugging Face User Access Token

Many of the popular Large Language Models (LLMs) in Hugging Face are [gated models](https://huggingface.co/docs/hub/en/models-gated). To access gated models, you need a Hugging Face [user access token](https://huggingface.co/docs/hub/en/security-tokens). Please create a Hugging Face user access token in your Hugging Face account, and set it below in `hf_token` variable below.

In [ ]:
import subprocess
import time
import os
import stat
import yaml
import json

hf_token=''
# Comment out next line if not using a Hugging Face gated model
assert hf_token, "Hugging Face user access token is required for gated models"
os.environ['HF_TOKEN']=hf_token

### Build Docker Containers

Next, we build the docker containers used to run the inference endpoint locally on this desktop.


In [ ]:
! source build-containers.sh

### Specify Hugging Face Model Id

Next, set the Hugging Face Model Id for the LLM you want to test in `hf_model_id` variable, below. You can specify the Hugging Face model id for a [Meta Llama](https://huggingface.co/meta-llama), [Deepseek](https://huggingface.co/deepseek-ai), or [Mistral AI](https://huggingface.co/mistralai) text-based Generative AI LLM. 

In [ ]:
hf_model_id = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B'
os.environ['MODEL_ID']=hf_model_id
os.environ['MAX_MODEL_LEN']=str(8192)
multi_modal_model=False

## Configure Tensor Parallel Size

Tensor parallel size depends on number of available device cores, and model size. We set tensor parallel size, by default, to minimum of number of available cores, or 8. Depeneding on the size of your model, you may adjust tensor parallel size , down or up, as needed. The number of backend inference servers is computed by dividing number of devices by tensor parallel size. For example, if there are 8 devices, and tensor parallel size is 2, 4 backend inference servers will be started.

In [ ]:
os.environ['TENSOR_PARALLEL_SIZE']=str(min(8, num_device))

### Specify Inference Server and Backend

Next, specify `inference_server`, and `backend` variables, below. This notebook supports [Triton Inference Server](https://github.com/triton-inference-server/server) with [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM), [vLLM](https://github.com/vllm-project/vllm), and [Transformers Neuronx in LMI engines](https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/tnx_user_guide.html) as backends. 

In [ ]:
inference_server = 'triton_inference_server'
assert inference_server in ['triton_inference_server', "djl_serving", "openai_server"]
backend = 'vllm'
assert backend in ['vllm', 'trtllm', 'tnx_lmi']

assert not multi_modal_model or (inference_server == "openai_server" and device=="cuda"), "Multi modal requires openai_server and cuda"
assert inference_server != "openai_server" or backend == "vllm", f"openai_server requires vllm backend"
assert backend != 'trtllm' or device == "cuda", f"TensorRT-LLM not supported on {device} device"
assert backend != 'tnx_lmi' or device == "neuron", f"Transformers-Neuronx-LMI is not supported on {device} device"

print(f"Using '{inference_server}' inference server with '{backend}' backend")

### Launch Inference Server

Next we use Docker compose to launch the inference server locally on this desktop.

In [ ]:
script_map = {
    "triton_inference_server": {
        "vllm": {
            "cuda": "../triton-server/vllm/compose-triton-vllm-cuda.sh",
            "neuron": "../triton-server/vllm/compose-triton-vllm-neuronx.sh"
        },
        "trtllm": {
            "cuda": "../triton-server/tensorrt-llm/compose-triton-tensorrt-llm.sh"
        },
        "tnx_lmi": {
            "neuron": "../triton-server/djl-lmi/compose-triton-djl-lmi-neuronx.sh"
        }
    },
    "djl_serving": {
        "vllm": {
            "cuda": "../djl-serving/vllm/compose-djl-lmi-vllm-cuda.sh",
            "neuron": "../djl-serving/vllm/compose-djl-lmi-vllm-neuronx.sh"
        },
        "trtllm": {
            "cuda": "../djl-serving/tensorrt-llm/compose-djl-lmi-tensorrt-llm.sh"
        },
        "tnx_lmi": {
            "neuron": "../djl-serving/transformers-neuronx/compose-djl-lmi-transformers-neuronx.sh"
        }
    },
    "openai_server": {
        "vllm": {
            "cuda": "../openai-server/vllm/compose-openai-server-vllm-cuda.sh",
            "neuron": "../openai-server/vllm/compose-openai-server-vllm-neuronx.sh"
        }
    }
}

script_path = script_map[inference_server][backend][device]
! {script_path} down
! {script_path} up

## Locust Testing

### Load Configuration

Below. we load the appropriate configuration file for the specified inference server, and backend.

In [ ]:
path = [ "config", f"{inference_server}-{backend}.yaml" ]
if multi_modal_model:
    path.insert(1, "multi-modal")

config_path=os.path.join(*path)
with open(config_path, "r") as mf:
    config=yaml.safe_load(mf)

print(json.dumps(config, indent=2))

### Verify inference server is up

The inference server may take several minutes to start up. Next, we verify the inference server is up. Do not proceed to next cell until inference server is up.

In [ ]:
import requests
try:
    response = requests.get(config['endpoint_url'])
    response_code = int(response.status_code)
    assert (response_code == 405) or (response_code == 424), f"Inference server is not yet up: {response_code}"
    print("Inference server is up!")
except:
    print("Inference server is not yet up")

### Validate Configuration

The configuration file specifies a prompt generator module. The module is dynamically loaded, and is invoked iteratively by the Locust endpoint user (see `endpoint_user.py`) to get next prompt to drive Locust testing.

Let us validate our configuration by making a single request and inspecting the response.

In [ ]:
from importlib import import_module
import re
from pprint import pprint
import sys

def get_prompt_generator():
    prompt_module_dir = config['module_dir']
    sys.path.append(prompt_module_dir)
    
    prompt_module_name = config['module_name']
    prompt_module=import_module(prompt_module_name)
    
    prompt_generator_name = config['prompt_generator']
    prompt_generator_class = getattr(prompt_module, prompt_generator_name)

    return prompt_generator_class()()
      
def fill_template(template: dict, template_keys:list, inputs:list) -> dict:
        
    assert len(template_keys) == len(inputs), f"template_keys: {template_keys}, prompts: {inputs}"
    for i, template_key in enumerate(template_keys):
        _template = template
        keys = template_key.split(".")
        for key in keys[:-1]:
            m = re.match(r'\[(\d+)\]', key)
            if m:
                key = int(m.group(1))
            _template = _template[key]

        _template[keys[-1]] = inputs[i]
    
    return template

def inference_request():
    prompt_generator = get_prompt_generator()
    inputs = next(prompt_generator)
    inputs = [inputs] if isinstance(inputs, str) else inputs

    template = config['template']
    assert template is not None

    template_keys = config['template_keys']
    assert template_keys is not None
    
    if "model" in template_keys:
        inputs.insert(0, hf_model_id)
    data = fill_template(template=template, template_keys=template_keys, inputs=inputs)

    body = json.dumps(data).encode("utf-8")
    pprint(body)
    headers = {"Content-Type":  "application/json"}
    response = requests.post(config['endpoint_url'], data=body, headers=headers)
    return response

response = inference_request()
assert int(response.status_code) == 200, f"Response status code {response.status_code} != 200"
pprint(f"Response Content: {response.json()}")

### Run Locust Load Testing

The Locust load testing below uses 32 users with 32 workers to drive concurrent load, and by default, is set to run for 60 seconds. You can adjust these values as needed. Keep `SPAWN_RATE` the same as `USERS` to drive maximum concurrency.

In [ ]:
ts = round(time.time() * 1000)

os.environ["MODEL"] = hf_model_id
os.environ["PROMPT_MODULE_DIR"] = config['module_dir']
os.environ["PROMPT_MODULE_NAME"] = config['module_name']
os.environ["PROMPT_GENERATOR_NAME"] = config['prompt_generator']
os.environ["TEMPLATE"] = json.dumps(config.get('template', {}))
os.environ["TEMPLATE_KEYS"] = json.dumps(config.get('template_keys', []))
os.environ["CONTENT_TYPE"]="application/json"
os.environ["ENDPOINT_NAME"] = config['endpoint_url']
os.environ["USERS"]="32"
os.environ["WORKERS"]="32"
os.environ["RUN_TIME"]="120s"
os.environ["SPAWN_RATE"]="32"
os.environ["SCRIPT"]="endpoint_user.py"
results_locust_path = os.path.join("output", "locust-testing")
os.environ["RESULTS_PREFIX"]=f"{results_locust_path}/results-{ts}"
    
try:
    with open("run_locust.log", "w") as logfile:
        print(f"Start Locust testing; logfile: run_locust.log; results: {results_locust_path}")
        path = os.path.join(os.getcwd(), "run_locust.sh")
        os.chmod(path, stat.S_IRUSR | stat.S_IEXEC)
        process = subprocess.Popen(path, encoding="utf-8", 
                                shell=True,stdout=logfile,stderr=subprocess.STDOUT)
        process.wait()
        logfile.flush()
        print(f"Locust testing completed")
except Exception as e:
    print(f"exception occurred: {e}")


## Visualize Locust Results

Below we first visualize the results of the Locust testing in a tabel. 

In [ ]:
import pandas as pd
from IPython.display import display
import numpy as np

results_path = os.environ["RESULTS_PREFIX"] + "_stats.csv"
df = pd.read_csv(results_path)
df = df.replace(np.nan, '')

top_n = 1
caption=f"Locust results"
df = df.truncate(after=top_n - 1, axis=0)
df = df.style \
      .format(precision=6) \
        .set_properties(**{'text-align': 'left'}) \
        .set_caption(caption)
display(df)

### Shutdown inference server

Next, we shutdown inference server.

In [ ]:
! {script_path} down